# Kullanıcı Tabanlı İş Birlikçi Filtreleme ( User-Based Collaborative Filtering )

- Kullanıcıların benzerlikler üzerinden öneriler yapar. 
- Örneğin Sinan bey'e film önerisinde bulunalım.Sinan beyin izleme alışkanlıklarına en benzer izleme geçmişine sahip başka bir kullanıcının izlediği filmlerin önerilmesini verebiliriz.
- 5 diğer kişi ve Sinan bey "ozark" filmini izlemiş ve beğenmiş olsun, 5 diğer kişi "Better call saul" filmini de izlemiş ve beğenmiş ancak Sinan bey bu filmi izlememiş. Bizim yapmak istediğimiz, Sinan beye "Better call saul" filmini önermek olacak. 

Bu metod item based collaborative filtering'e benzemektedir. Item based collaborative filtering'de; bir filmin benzer beğenilme davranışlarına sahip diğer filmler ile olan durumlarını değerlendirdik. Bir film ile en çok beğenilme pattern'ına (örüntüsüne) sahip olan filmleri önermiştik.


Ancak item based collaborative filtering'den farkı, kullanıcılara daha çok odaklanılmasıdır. Örnek olarak A kişisinin hangi filmleri izlediği bulunmaktadır. Daha sonrasında A kişisinin izlediği filmlere benzer film izleyen kişiler bulunmaktadır. Bunun yanı sıra A kişisi ile benzer filmleri izleyen kişiler içerisinden de "A kişisi ile en benzer beğenme veya beğenmeme davranışı gösteren kişiler" tespit edilecektir.

Yani bu A kişisi bir film beğenmiş ise diğerlerinin de beğendiği,  filmler tespit edilecektir.

In [1]:
# User-Based Collaborative Filtering

# Adım 1: Veri Setinin Hazırlanması
# Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
# Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
# Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
# Adım 5: Weighted Average Recommendation Score'un Hesaplanması
# Adım 6: Çalışmanın Fonksiyonlaştırılması

### Adım 1: Veri Setini Hazırlama

In [1]:
import pandas as pd 
pd.set_option('display.width', 500)

In [2]:
movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')

In [4]:
df= movie.merge(rating, how="left", on="movieId")
df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,2015-03-30 19:32:59
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,2015-03-30 19:48:08
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,2015-03-30 19:56:32
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,2015-03-30 19:57:46


In [5]:
comment_counts= pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [6]:
rare_movies= comment_counts[comment_counts["title"] <= 1000].index

In [7]:
common_movies = df[~df["title"].isin(rare_movies)]
common_movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


In [8]:
user_movie_df= common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df